<div style="padding:15px 20px 20px 20px;border-left:3px solid #87CEFA;background-color:#f0fbff;border-radius: 20px;color:#424242;">

# **Notebook Description**

## Table of Contents
- **1. Setup**
    - 1.1) Local Setup
    - 1.2) Downloading the Data

- **2. Analysis on the channel dataset**
    - 2.1) 
    - 2.2) 
    
- **3. Analysis on the video metadata dataset**
    - 3.1) 
    - 3.2)

- **4. Analysis on the comment dataset**
    - 4.1) 
    - 4.2)

- **5.**
    
</div>

# **1. Setup**

## 1.1) Local Setup

In [1]:
# !pip install -r pip_requirements.txt

In [11]:
import os
import src.utils.data_utils as data_utils

In [12]:
ROOT_PATH = os.getcwd()
os.listdir(ROOT_PATH)

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'data',
 'pip_requirements.txt',
 'README.md',
 'results.ipynb',
 'src',
 'test Notebooks',
 'tests']

## 1.2) Downloading the Data

In [13]:
# Download the datasets (ordered by size), comment out the ones you don't need 
datasets = [
    ("https://zenodo.org/records/4650046/files/_raw_df_channels.tsv.gz?download=1", "channels.tsv.gz"), # 6.0 MB
    # ("https://zenodo.org/records/4650046/files/_raw_df_timeseries.tsv.gz?download=1", "time_series.tsv.gz" ), # 571.1 MB
    # ("https://zenodo.org/records/4650046/files/num_comments.tsv.gz?download=1", "num_comments.tsv.gz"), # 754.6 MB
    # ("https://zenodo.org/records/4650046/files/num_comments_authors.tsv.gz?download=1", "num_comments_authors.tsv.gz"), # 1.4 GB
    # ("https://zenodo.org/records/4650046/files/yt_metadata_en.jsonl.gz?download=1", "yt_metadata.jsonl.gz"), # 13.6 GB
    # ("https://zenodo.org/records/4650046/files/youtube_comments.tsv.gz?download=1", "youtube_comments.tsv.gz") # 77.2 GB
]

data_utils.get_data(datasets)

channels.tsv.gz downloaded successfully.


# 2. Analysis on the channel dataset

# 3. Analysis on the video metadata dataset

# 4. Analysis on the comment dataset

# 5. 